Dataframes Needed:
- Live games (Teams playing, scores, results)
- Box scores (Player stats)
- Lead tracker

Home Page
- Live game scores, Youtube videos of highlights

Game Page
- Box scores
- Matchup stats (lead tracker, team %s)
- Youtube highlight

Visualizations
- [Blank]

In [76]:
import nbapy
import datetime
import warnings
import numpy as np
import pandas as pd
import dateutil.parser
from nbapy import game
from nbapy.scoreboard import Scoreboard

pd.set_option('max_columns', None)
pd.options.mode.chained_assignment = None

In [113]:
test_gameid = '0021901308'
test_date = '08-13-2020'

In [3]:
import constants

### Get Games Function

In [108]:
def get_games(datestring):
    """
    Input:
        datestring (str) - date of NBA games to grab
    
    Output:
        games (pandas df) - dataframe of NBA games, results, and teams playing
        
    """
    date = dateutil.parser.parse(datestring)
    scoreboard = Scoreboard(date.month, date.day, date.year)
    
    # Grab today's game information
    game_header = scoreboard.game_header()
    line_score = scoreboard.line_score()
    
    # Simplify columns
    games = game_header[['GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_TEXT', 
                         'LIVE_PERIOD', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
                         'NATL_TV_BROADCASTER_ABBREVIATION']]
    score = line_score[['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']]
    
    # Dictionary of game values
    team_info = score.set_index('TEAM_ID')['TEAM_ABBREVIATION'].to_dict()
    team_pts = score.set_index('TEAM_ID')['PTS'].to_dict()
    
    # Map to games info
    games['HOME_TEAM_ABBREVIATION'] = games.HOME_TEAM_ID.map(team_info)
    games['VISITOR_TEAM_ABBREVIATION'] = games.VISITOR_TEAM_ID.map(team_info)
    games['HOME_TEAM_PTS'] = games.HOME_TEAM_ID.map(team_pts)
    games['VISITOR_TEAM_PTS'] = games.VISITOR_TEAM_ID.map(team_pts)

    # If games have not started yet, replace the None value with 0
    for i, row in games.iterrows():
        if row['HOME_TEAM_PTS'] is None:
            games['HOME_TEAM_PTS'].fillna(value=0, inplace=True)
            games['VISITOR_TEAM_PTS'].fillna(value=0, inplace=True)
    
    # Create a winner column for web layout
    winners = []
    for i, value in games.iterrows():
        if value["HOME_TEAM_PTS"] > value["VISITOR_TEAM_PTS"]:
            winners.append(value["HOME_TEAM_ABBREVIATION"])
        elif value["HOME_TEAM_PTS"] < value["VISITOR_TEAM_PTS"]:
            winners.append(value["VISITOR_TEAM_ABBREVIATION"])
        else:
            winners.append('None')
    
    games['WINNER'] = winners
    
    return games

games = get_games(test_date)
games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,1,0021901308,Final,4,1610612738,1610612764,None,BOS,WAS,90,96,WAS
1,2,0021901310,Final,4,1610612747,1610612758,None,LAL,SAC,122,136,SAC
2,3,0021901311,Final,4,1610612763,1610612749,None,MEM,MIL,119,106,MEM
3,4,0021901313,Final,4,1610612756,1610612742,TNT,PHX,DAL,128,102,PHX
4,5,0021901314,Final,4,1610612762,1610612759,TNT,UTA,SAS,118,112,UTA
5,6,0021901309,Final,4,1610612751,1610612757,TNT,BKN,POR,133,134,POR
6,7,0021901312,Final,4,1610612753,1610612740,None,ORL,NOP,133,127,ORL


### Get Live Games

Columns Needed
- GAME_STATUS_TEXT
- HOME_TEAM_ABBREVIATION
- VISITOR_TEAM_ABBREVIATION
- HOME_TEAM_PTS
- VISITOR_TEAM_PTS

In [9]:
today = datetime.datetime.now()
today_string = today.strftime("%m-%d-%Y")
today_string

'08-25-2020'

In [11]:
yday = '08-24-2020'
yday

'08-24-2020'

In [19]:
# live_games = get_games(today_string)
# live_games

In [109]:
def get_livegames():
    """
    Input:
        None
        
    Output:
        live_games (pandas df) - Dataframe showing results for today's NBA games
    """
    today = datetime.datetime.now()
    today_string = today.strftime("%m-%d-%Y")
    live_games = get_games(today_string)
    live_games = live_games.fillna(0)
    live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]] = live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]].astype(int)
    return live_games

live_games = get_livegames()
live_games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,3,0041900165,6:30 pm ET,0,1610612743,1610612762,TNT,DEN,UTA,0,0,None
1,4,0041900155,9:00 pm ET,0,1610612746,1610612742,TNT,LAC,DAL,0,0,None


In [99]:
live_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 11 columns):
GAME_SEQUENCE                       2 non-null int64
GAME_ID                             2 non-null object
GAME_STATUS_TEXT                    2 non-null object
LIVE_PERIOD                         2 non-null int64
HOME_TEAM_ID                        2 non-null int64
VISITOR_TEAM_ID                     2 non-null int64
NATL_TV_BROADCASTER_ABBREVIATION    2 non-null object
HOME_TEAM_ABBREVIATION              2 non-null object
VISITOR_TEAM_ABBREVIATION           2 non-null object
HOME_TEAM_PTS                       0 non-null object
VISITOR_TEAM_PTS                    0 non-null object
dtypes: int64(4), object(7)
memory usage: 304.0+ bytes


In [104]:
for i, row in live_games.iterrows():
    if row['HOME_TEAM_PTS'] is None:
        live_games['HOME_TEAM_PTS'].fillna(value=0, inplace=True)
        live_games['VISITOR_TEAM_PTS'].fillna(value=0, inplace=True)

In [105]:
live_games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS
0,3,0041900165,6:30 pm ET,0,1610612743,1610612762,TNT,DEN,UTA,0,0
1,4,0041900155,9:00 pm ET,0,1610612746,1610612742,TNT,LAC,DAL,0,0


### Get Box Scores

In [121]:
def get_boxscore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull box scores
    
    Output:
        box_score (pandas df) - Dataframe containing the box score results of Game ID
        
    """
    boxscore = game.BoxScore(game_id).players_stats()
    
    # Data cleaning
    boxscore = boxscore.fillna(0)
    float_scores = boxscore.select_dtypes('float')
    headers = boxscore.columns.values
    
    # Keep PCT as floats
    pct_df = float_scores.filter(regex='PCT')
    pct_cols = pct_df.columns.values
    boxscore.drop(pct_cols, axis=1, inplace=True)
    boxscore = pd.concat([boxscore, pct_df], axis=1)
    
    # Convert stats to ints
    int_df = float_scores.astype(int).drop(pct_cols, axis=1)
    int_cols = int_df.columns.values
    boxscore.drop(int_cols, axis=1, inplace=True)
    boxscore = pd.concat([boxscore, int_df], axis=1)
    
    # Keep original order of headers
    boxscore = boxscore[headers]
    team1, team2 = boxscore.groupby("TEAM_ID")
    
    # Game summary
    summary = game.Info(game_id).game_summary()
    
    return team1[1], team2[1], summary

team1, team2, summary = get_boxscore(today_game)
team1

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
13,0041900165,1610612743,DEN,Denver,203924,Jerami Grant,F,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0041900165,1610612743,DEN,Denver,200794,Paul Millsap,F,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0041900165,1610612743,DEN,Denver,203999,Nikola Jokic,C,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0041900165,1610612743,DEN,Denver,1628420,Monte Morris,G,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0041900165,1610612743,DEN,Denver,1627750,Jamal Murray,G,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0041900165,1610612743,DEN,Denver,1628470,Torrey Craig,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0041900165,1610612743,DEN,Denver,1629008,Michael Porter Jr.,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,0041900165,1610612743,DEN,Denver,1628408,PJ Dozier,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,0041900165,1610612743,DEN,Denver,203486,Mason Plumlee,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,0041900165,1610612743,DEN,Denver,1628966,Keita Bates-Diop,,DNP - Coach's Decision,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
game.BoxScore('0041900165').players_stats()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS


In [118]:
today_game = '0041900165'
today_game

'0041900165'

### Get Team Stats

In [ ]:
def get_teamstats(game_id):
    """
    Input:
        game_id (str) - Game ID to pull team stats
        
    Output:
        team_stats (pandas df) - Dataframe showing the teams percentages
    
    """
    team_stats = game.BoxScore(game_id).team_stats()
    
    return team_stats

team_stats = get_teamstats(test_gameid)
team_stats

#### Data Visualizations

In [ ]:
import plotly.graph_objects as go

labels = ['Field Goals Attempted', 'Field Goals Made']
values = [45, 45]

layout = ['Field Goal Percentage']

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.7,
                             marker=dict(colors=["#D3D3D3", "#228B22"]))])
fig.show()

In [ ]:
import plotly.offline as ofl
ofl.init_notebook_mode()

fig = {

  # key is data and value is a list of objects - one for each data trace
  "data": [
    {
      # defining values
      "values": [45, 45],

      # defining labels
      "labels": ['Field Goals Attempted', 'Field Goals Made'],

      # Information to be displayed on hover
      "hoverinfo":"label+percent+name",

      # hole which creates the donut
      "hole": .5,

      # type of chart is defined here
      "type": "pie"
    }],
  "layout": {
        "title":"Field Goal Percentage",
        "text":["45/90"],
        "annotations": [
            {
                "font": {"size": 20},
                "showarrow": False,
                "text": "45/90",
                "x": 0.20,
                "y": 0.5
            },
        ]
    }
}

ofl.iplot(fig, filename='donut')

### Get Lead Tracker

In [ ]:
def get_linescore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull line score
    
    Ouput:
        line_score (pandas df) - Dataframe showing the points by quarter
    
    """
    line_score = game.Info(game_id).line_score()
    return line_score

line_score = get_linescore(test_gameid)
line_score

### Get Standings

In [ ]:
scoreboard = Scoreboard()
east_standings = scoreboard.east_conf_standings_by_day()
west_standings = scoreboard.west_conf_standings_by_day()

In [ ]:
west_standings

### Workspace

In [ ]:
test_list = ['08/20/2020']
test_list

In [ ]:
test_list[0]